# Housing Prices Competition

#### What we used:
* "Decision Tree Regression" as model
* "MAE" as metric
* "train_test_split" to trunk data to train/validate/test parts
* "LabelEncoder" to decode object features to digit

In [1]:
pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [2]:
# The main module for manipulating with the data:
import pandas as pd

In [3]:
# The main familiar using model:
from sklearn.tree import DecisionTreeRegressor
# To encode categories parameters:
from sklearn.preprocessing import LabelEncoder
# The main metric for model:
from sklearn.metrics import mean_absolute_error
# The method for trunking data to train and validate parts:
from sklearn.model_selection import train_test_split

In [4]:
# The paths of data:
test_path = "../test_data.csv"
train_path = "../train_data.csv"

# Clone data to dataframe:
test_dataframe = pd.read_csv(test_path)
train_dataframe = pd.read_csv(train_path)

# Check columns:
train_dataframe.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [5]:
# Take important features (a first view)
# we remove all subjective parameters by type qualities:
features = [
            "LotArea", "Utilities", "YearBuilt", "Exterior1st", "Foundation", "Heating", "CentralAir", "1stFlrSF", 
            "2ndFlrSF", "BedroomAbvGr", "FullBath", "KitchenAbvGr", "TotRmsAbvGrd", "Fireplaces", "GarageArea", 
            "OpenPorchSF", "EnclosedPorch", "PoolArea"
           ]

# Take target feature:
Y = train_dataframe.SalePrice

# Take samples with necessary data columns:
necessary_train_data = train_dataframe[features]

# Check that we don't have null rows:
necessary_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   LotArea        1460 non-null   int64 
 1   Utilities      1460 non-null   object
 2   YearBuilt      1460 non-null   int64 
 3   Exterior1st    1460 non-null   object
 4   Foundation     1460 non-null   object
 5   Heating        1460 non-null   object
 6   CentralAir     1460 non-null   object
 7   1stFlrSF       1460 non-null   int64 
 8   2ndFlrSF       1460 non-null   int64 
 9   BedroomAbvGr   1460 non-null   int64 
 10  FullBath       1460 non-null   int64 
 11  KitchenAbvGr   1460 non-null   int64 
 12  TotRmsAbvGrd   1460 non-null   int64 
 13  Fireplaces     1460 non-null   int64 
 14  GarageArea     1460 non-null   int64 
 15  OpenPorchSF    1460 non-null   int64 
 16  EnclosedPorch  1460 non-null   int64 
 17  PoolArea       1460 non-null   int64 
dtypes: int64(13), object(5)
memo

In [6]:
# Check all values of categorical columns:
necessary_train_data.Utilities.unique()

array(['AllPub', 'NoSeWa'], dtype=object)

In [7]:
necessary_train_data.CentralAir.unique()

array(['Y', 'N'], dtype=object)

In [8]:
necessary_train_data.Exterior1st.unique()

array(['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing',
       'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn',
       'Stone', 'ImStucc', 'CBlock'], dtype=object)

In [9]:
necessary_train_data.Foundation.unique()

array(['PConc', 'CBlock', 'BrkTil', 'Wood', 'Slab', 'Stone'], dtype=object)

In [10]:
necessary_train_data.Heating.unique()

array(['GasA', 'GasW', 'Grav', 'Wall', 'OthW', 'Floor'], dtype=object)

In [11]:
# Encoder other parameters (don't use dummies because for model will be better use label encoder):
label_encoder = LabelEncoder()
necessary_train_data[["Exterior1st", "Foundation", "Heating", "Utilities", "CentralAir"]] = \
    necessary_train_data[[
                            "Exterior1st", 
                            "Foundation", 
                            "Heating", 
                            "Utilities", 
                            "CentralAir"
                        ]].apply(lambda x: label_encoder.fit_transform(x))

# Check final data:
necessary_train_data.head()

C:\Users\Deniz\AppData\Local\Temp\ipykernel_5724\3572952141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  necessary_train_data[["Exterior1st", "Foundation", "Heating", "Utilities", "CentralAir"]] = \


,LotArea,Utilities,YearBuilt,Exterior1st,Foundation,Heating,CentralAir,1stFlrSF,2ndFlrSF,BedroomAbvGr,FullBath,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,OpenPorchSF,EnclosedPorch,PoolArea
0,8450,0,2003,12,2,1,1,856,854,3,2,1,8,0,548,61,0,0
1,9600,0,1976,8,1,1,1,1262,0,3,2,1,6,1,460,0,0,0
2,11250,0,2001,12,2,1,1,920,866,3,2,1,6,1,608,42,0,0
3,9550,0,1915,13,0,1,1,961,756,3,1,1,7,1,642,35,272,0
4,14260,0,2000,12,2,1,1,1145,1053,4,2,1,9,1,836,84,0,0


In [12]:
# Split data for fitting and validating:
train_X, validate_X, train_Y, validate_Y = train_test_split(necessary_train_data, Y, test_size=0.2, random_state=1)

In [13]:
def get_depth(number_of_depth: int):
    decision_tree_model = DecisionTreeRegressor(criterion="absolute_error", max_depth=number_of_depth, random_state=1)
    
    decision_tree_model.fit(train_X, train_Y)
    
    predictions = decision_tree_model.predict(validate_X)
    
    return mean_absolute_error(predictions, validate_Y)

def get_max_leaf_nodes(number_max_leaf_nodes: int, depth: int):
    decision_tree_model = DecisionTreeRegressor(criterion="absolute_error",  max_depth=depth, 
                                                max_leaf_nodes=number_max_leaf_nodes, random_state=1)
    
    decision_tree_model.fit(train_X, train_Y)
    
    predictions = decision_tree_model.predict(validate_X)
    
    return mean_absolute_error(predictions, validate_Y)

# To get the best max depth and max leaf counter:
def get_the_best_tree_parameters():
    variants_of_depth = {depth: get_depth(depth) for depth in range(5, 300)}
    best_depth = min(variants_of_depth, key=variants_of_depth.get)
    variants_of_leaf = {leaf: get_max_leaf_nodes(leaf, best_depth) for leaf in range(3, 50)}
    
    return (best_depth, min(variants_of_leaf, key=variants_of_leaf.get))

In [ ]:
# The best params from training data:
best_parameters = get_the_best_tree_parameters()

print(best_parameters)

In [ ]:
# We found the best values of parameters for decision tree, then we can use for training all samples:
final_training_samples = pd.concat([train_X, validate_X], axis=0)
final_training_targets = pd.concat([train_Y, validate_Y], axis=0)

# Check dataframe of samples for example:
final_training_samples

In [ ]:
# Create the best model for our data:
decision_tree_model = DecisionTreeRegressor(
                                            criterion="absolute_error", 
                                            max_depth=best_parameters[0], 
                                            max_leaf_nodes=best_parameters[1], 
                                            random_state=1
                                           )

# Make the final fitting:
decision_tree_model.fit(final_training_samples, final_training_targets)

# Add Id for future submission:
features.append("Id")

# Prepare test data:
test_samples = test_dataframe[features]

# Check that we don't have null values:
test_samples.info()

In [ ]:
# Calculate mean value for null cells:
mean_price = int((test_samples["GarageArea"].mean()))

# Check the getting value:
print(mean_price)

In [ ]:
# Fill null cell by mean value of parameter (we have only 1 string):
test_samples.GarageArea.fillna(mean_price, inplace=True)

# Prepare data:
test_samples[["Exterior1st", "Foundation", "Heating", "Utilities", "CentralAir"]] = \
    test_samples[[
                    "Exterior1st", 
                    "Foundation", 
                    "Heating", 
                    "Utilities", 
                    "CentralAir"
                ]].apply(lambda x: label_encoder.fit_transform(x))

# Check result test data:
test_samples

In [ ]:
# The submissions Ids:
submission_ids = test_samples.Id

# Drop Id column for fitting:
test_samples.drop(columns=["Id"], axis=1, inplace=True)

# Make the final prediction:
test_prediction = decision_tree_model.predict(test_samples)

# Check the final price predictions:
test_prediction

In [ ]:
# Create submission dataframe:
submisson_file = pd.DataFrame(
                                {
                                    "Id": submission_ids,
                                    "SalePrice": test_prediction
                                }
                             )

# Check the result:
submisson_file

In [ ]:
# Create a submission file:
submisson_file.to_csv("submission.csv", index=False)